## 1. Exploratory Data Analysis (EDA)

In [12]:
import pandas as pd 
import numpy as np 

In [ ]:
import json

with open("C:\\Users\\lamia\\Downloads\\course_data.json", "r", encoding="utf-8") as f:
    data = json.load(f) #قرأت الملف

type(data), len(data)  #فحصت نوعه وحجمه


(dict, 8479)

الملف عبارة عن قاموس يحتوي على 8479 مقرر

In [ ]:
all_reviews = [] #مريت على كل مقرر 

for course, info in data.items(): # دخلت على قائمة التقييمات داخله
    for r in info["reviews"]:
        if r["course_rating"] is not None:  # نشيل القيم الفارغة
            all_reviews.append({ 
                "course": course,
                "text": r["review_text"],
                "label": r["course_rating"]
            })

len(all_reviews) #جمعت كل الريفيو في قائمة وحدة 


14616

يعني عندي 14616 تقييم صالح للتحليل

In [ ]:
import pandas as pd

df = pd.DataFrame(all_reviews) #حولت البيانات الى داتافريم 
df.head() # عرضت اول 5 صفوف للتأكد من الشكل

,course,text,label
0,CS 115,go to office hours and practice,liked course
1,CS 115,One of my least favourite courses. Although th...,disliked course
2,CS 115,It starts with a very low pace but after midte...,disliked course
3,CS 115,Took this in 2018 with no programming experien...,liked course
4,CS 115,I loved everything about cs 115. Great instruc...,liked course


In [ ]:
df["label"] = df["label"].map({
    "liked course": "Good",
    "disliked course": "Bad"
}) # حولت القيم النصية الى تصنيفات موحدة 

df["label"].value_counts() # حسبت عدد كل فئة


label
Good    9786
Bad     4830
Name: count, dtype: int64

In [ ]:
# تحليل طول التقييمات 
df["review_length"] = df["text"].str.split().str.len()  # حسبت عدد الكلمات في كل تقييم 
df["review_length"].describe()  # استخرجت احصائيات وصفية 

count    14616.000000
mean        39.694376
std         42.180647
min          0.000000
25%         13.000000
50%         27.000000
75%         51.000000
max        760.000000
Name: review_length, dtype: float64

المتوسط ≈ 40 كلمة
الوسيط = 27 كلمة
أقصى طول = 760 كلمة

## 2. Text Preprocessing

In [20]:
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS  # قائمة جاهزة للكلمات الشائعة بالإنجليزية

stop_words = set(ENGLISH_STOP_WORDS)  # تحويلها إلى set لتسريع البحث والحذف

def clean_text(t: str) -> str:
    t = t.lower()                     # تحويل النص إلى حروف صغيرة لتوحيد الصيغة
    t = re.sub(r"[^a-z\s]", " ", t)   # إزالة علامات الترقيم، الأرقام، والرموز الخاصة
    t = re.sub(r"\s+", " ", t).strip()# إزالة المسافات الزائدة وتنظيف النص
    tokens = [w for w in t.split() if w not in stop_words]  # حذف الكلمات الشائعة غير المفيدة
    return " ".join(tokens)           # إعادة دمج الكلمات في نص واحد نظيف

df["clean_text"] = df["text"].astype(str).apply(clean_text)  # تطبيق دالة التنظيف على كل التقييمات
df[["text","clean_text"]].head()     # عرض النص الأصلي مقابل النص المنظف للمقارنة


,text,clean_text
0,go to office hours and practice,office hours practice
1,One of my least favourite courses. Although th...,favourite courses things nicely organized rack...
2,It starts with a very low pace but after midte...,starts low pace midterm gets really fast under...
3,Took this in 2018 with no programming experien...,took programming experience started pretty slo...
4,I loved everything about cs 115. Great instruc...,loved cs great instructors fair assignments tr...


## 3. Tokenization